<a href="https://colab.research.google.com/github/gabrielflorianoo/Atividades-Propostas-Web-Front-End/blob/main/Bone_Fracture_Multi_Region_X_ray_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ! kaggle datasets download 'bmadushanirodrigo/fracture-multi-region-x-ray-data'
# ! unzip 'fracture-multi-region-x-ray-data.zip'

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing import image_dataset_from_directory

In [ ]:
from struct import unpack
from tqdm import tqdm
import os


marker_mapping = {
    0xffd8: "Start of Image",
    0xffe0: "Application Default Header",
    0xffdb: "Quantization Table",
    0xffc0: "Start of Frame",
    0xffc4: "Define Huffman Table",
    0xffda: "Start of Scan",
    0xffd9: "End of Image"
}


class JPEG:
    def __init__(self, image_file):
        with open(image_file, 'rb') as f:
            self.img_data = f.read()

    def decode(self):
        data = self.img_data
        while(True):
            marker, = unpack(">H", data[0:2])
            # print(marker_mapping.get(marker))
            if marker == 0xffd8:
                data = data[2:]
            elif marker == 0xffd9:
                return
            elif marker == 0xffda:
                data = data[-2:]
            else:
                lenchunk, = unpack(">H", data[2:4])
                data = data[2+lenchunk:]
            if len(data)==0:
                break


bads = []

root_img = '/content/Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/train/fractured'
for img in os.listdir(root_img):
    image = os.path.join(root_img, img)
    image = JPEG(image)
    try:
        image.decode()
    except:
        bads.append(img)

print(bads)

for name in bads:
    os.remove(os.path.join(root_img,name))

[]


In [ ]:
# Load training dataset excluding corrupted images
train = image_dataset_from_directory(
    '/content/Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/train',
    image_size=(256, 256),
    validation_split=0.2,  # Adjust this split according to your dataset
    subset='training',
    seed=222
)

# Load validation dataset excluding corrupted images
val = image_dataset_from_directory(
    '/content/Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/val',
    image_size=(256, 256),
    validation_split=0.2,  # Adjust this split according to your dataset
    subset='validation',
    seed=222
)


Found 9236 files belonging to 2 classes.
Using 7389 files for training.
Found 819 files belonging to 2 classes.
Using 163 files for validation.


In [ ]:
class_names = train.class_names
num_classes = len(class_names)

In [ ]:
train_iter = iter(train)
imgs, labels = next(train_iter)

In [ ]:
# fig, ax = plt.subplots(3, 3, figsize=(14, 8))

# ax = ax.flatten()
# idx = 0
# for img, ax in zip(imgs, ax):
#     ax.imshow(img / 255)
#     ax.set_title(class_names[np.argmax((labels[idx]))])
#     idx += 1

# plt.subplots_adjust(wspace=0.5, hspace=0.5)
# plt.show()

In [ ]:
# Caching
AUTOTUNE = tf.data.AUTOTUNE

train = train.cache().shuffle(1000).prefetch(AUTOTUNE)
val = val.cache().prefetch(AUTOTUNE)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

In [ ]:
model = Sequential([
    layers.Rescaling(input_shape=(256, 256, 3), scale=1./255),
    layers.Conv2D(16, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.2),
    layers.Flatten(),
    layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=['accuracy']
)

In [ ]:
model.fit(train, validation_data=val, epochs=10, verbose=1)

Epoch 1/10
231/231 [==============================] - 89s 56ms/step - loss: 0.4163 - accuracy: 0.8036 - val_loss: 0.3150 - val_accuracy: 0.8834
Epoch 2/10
231/231 [==============================] - 12s 54ms/step - loss: 0.1571 - accuracy: 0.9479 - val_loss: 0.2402 - val_accuracy: 0.9141
Epoch 3/10
231/231 [==============================] - 12s 54ms/step - loss: 0.0768 - accuracy: 0.9765 - val_loss: 0.1729 - val_accuracy: 0.9571
Epoch 4/10
231/231 [==============================] - 12s 54ms/step - loss: 0.0420 - accuracy: 0.9884 - val_loss: 0.1749 - val_accuracy: 0.9632
Epoch 5/10
231/231 [==============================] - 12s 54ms/step - loss: 0.0257 - accuracy: 0.9931 - val_loss: 0.1467 - val_accuracy: 0.9755
Epoch 6/10
231/231 [==============================] - 12s 54ms/step - loss: 0.0192 - accuracy: 0.9931 - val_loss: 0.1360 - val_accuracy: 0.9571
Epoch 7/10
231/231 [==============================] - 12s 54ms/step - loss: 0.0209 - accuracy: 0.9926 - val_loss: 0.0968 - val_accuracy:

In [ ]:
pred = model.predict(imgs)

print(class_names)

idx = 0
for x in pred:
    rd = round(x[0])
    print(class_names[rd], 'real labels: ', labels[idx])
    idx += 1

1/1 [==============================] - 0s 36ms/step
['fractured', 'not fractured']
fractured real labels:  tf.Tensor(0, shape=(), dtype=int32)
fractured real labels:  tf.Tensor(0, shape=(), dtype=int32)
not fractured real labels:  tf.Tensor(1, shape=(), dtype=int32)
fractured real labels:  tf.Tensor(0, shape=(), dtype=int32)
not fractured real labels:  tf.Tensor(1, shape=(), dtype=int32)
not fractured real labels:  tf.Tensor(1, shape=(), dtype=int32)
fractured real labels:  tf.Tensor(0, shape=(), dtype=int32)
not fractured real labels:  tf.Tensor(1, shape=(), dtype=int32)
fractured real labels:  tf.Tensor(0, shape=(), dtype=int32)
fractured real labels:  tf.Tensor(0, shape=(), dtype=int32)
not fractured real labels:  tf.Tensor(1, shape=(), dtype=int32)
fractured real labels:  tf.Tensor(0, shape=(), dtype=int32)
fractured real labels:  tf.Tensor(0, shape=(), dtype=int32)
not fractured real labels:  tf.Tensor(1, shape=(), dtype=int32)
not fractured real labels:  tf.Tensor(1, shape=(), dt